In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np 
import os
import requests
from gensim.downloader import load
import dask

# Loading in memory

In [151]:
def load_data_in_memory():
    """
    Load dataset from hugging face servers in memory
    """
    url = "https://datasets-server.huggingface.co/parquet?dataset=aadityaubhat%2FGPT-wiki-intro"
    response = requests.get(url)
    if response.status_code !=200:
        return f"error during dataset request: {response.status_code}"
    
    url_parquet = [files['url'] for files in response.json()['parquet_files']]
    
    
    df = [pd.read_parquet(url_) for url_ in url_parquet]
    
    return pd.concat(df)

In [152]:
def get_ds(path_data):
    """
    Load dataset and basic transformation for our task
    """
    if 'data.csv' in os.listdir(path_data):
        print('Loading dataset from local...')
        df = pd.read_csv(os.path.join(path_data,'data.csv'),index_col='id')
    else:
        df = load_data_in_memory()
        df.set_index('id',inplace=True)
        df['random']=np.random.random(len(df))

        # reorganize ds and randomize samples wiki/generated
        df.loc[df['random']<.5,'text']=df['generated_intro']
        df.loc[df['random']<.5,'label']='generated'
        df.loc[df['random']>=.5,'text']=df['wiki_intro']
        df.loc[df['random']>=.5,'label']='wiki'
        # dump csv
        df.to_csv(os.path.join(path_data,'data.csv'))
        
    df['label']=df['label'].replace({'generated':1,'wiki':0})
    
    display(df.head())
    
    return df

In [153]:
PATH_DATA = os.path.join(os.path.dirname(os.path.abspath(os.path.curdir)),'raw_data')

df = get_ds(PATH_DATA)

Loading dataset from local...


,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens,random,text,label
id,,,,,,,,,,,,,,
63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88,0.170068,Sexhow railway station was a railway station l...,1
279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101,0.839452,"In Finnish folklore, all places and things, an...",0
287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65,0.532203,"In mathematics, specifically differential calc...",0
26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150,0.715507,is a Japanese shōjo manga series written and i...,0
38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162,0.395063,"Robert Milner ""Rob"" Bradley, Jr. (born August ...",1


In [17]:
len(df)

150000

In [155]:
# compute # sentences
df['nsentences'] = df['text'].apply(lambda x : len(x.split('.')))

In [156]:
def word_per_sentence(text):
    """
    Compute the mean and variance of the number of words per sentences of a text.
    """
    sentences = text.split('.')
    lengths = []
    for s in sentences : 
        lengths.append(len(s.split()))
    return [np.mean(np.array(lengths)),np.std(np.array(lengths))]

In [157]:
df_e = pd.concat([
    df,
    df['text'].apply(word_per_sentence)\
                .apply(pd.Series)\
                .rename({0:'mean_w_p_s',1:'var_w_p_s'},axis=1)]
                 ,axis=1)

In [158]:
X = df_e[['text','nsentences','mean_w_p_s','var_w_p_s']]
y = df_e[['label']]

# Serialize dataset

In [10]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _nonscaler_feature(value):
    serialized_nonscalar = tf.io.serialize_tensor(value)
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[serialized_nonscalar.numpy()]))

In [11]:
# print(_bytes_feature(b'test_string'))
# print(_bytes_feature(u'test_bytes'.encode('utf-8')))

# print(_float_feature(np.exp(1)))

# print(_nonscaler_feature([1,2,3]))

# print(_int64_feature(True))
# print(_int64_feature(1))

In [12]:
def serialize_example(id_,text,vect, nsentences, mean_w_p_s ,var_w_p_s, label):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible
    # data type.
    feature = {
        'id': _int64_feature(int(id_)),
        'text': _bytes_feature(text),
        'vect':_nonscaler_feature(vect),
        'nsentences': _int64_feature(int(nsentences)),
        'mean_w_p_s': _float_feature(float(mean_w_p_s)),
        'var_w_p_s': _float_feature(float(var_w_p_s)),
        'label': _int64_feature(int(label)),
    }

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# add word2vec

In [14]:
%%time
for i in range(20000):
    if 'hello' in wv: 
        wv.get_vector('hello')

NameError: name 'wv' is not defined

In [ ]:
%%time
for i in range(20000):
    if 'hello' in wv.index_to_key: 
        wv.get_vector('hello')

In [15]:
# sentences = X.text[:100]  
# LEN=len(sentences)
# vect=[]
# for i,x in enumerate(sentences):
#     vect.append(embed_sentence_pretrained(wv,x))
#     if i%50==0:
#         print(f'{i*100/LEN:.2f}%')

In [39]:
@dask.delayed
def embed_sentence_pretrained(w2v, sentence):
    """
    Embed a sentence given a trained Word2Vec
    """
    embedded_sentence = []
    for word in sentence:
        if word in w2v:
            embedded_sentence.append(w2v.get_vector(word))

    return np.array(embedded_sentence)

In [48]:
def embed_sentence_pretrained(w2v, sentence):
    """
    Embed a sentence given a trained Word2Vec
    """
    embedded_sentence = []
    for word in sentence:
        if word in w2v:
            embedded_sentence.append(w2v.get_vector(word))

    return np.array(embedded_sentence)

def embed_corpus(X,wv):
    sentences = X.text
    LEN = len(sentences)
    vect = []
    for i, x in enumerate(sentences):
        vect.append(embed_sentence_pretrained(wv, x))
#         if i % 100 == 0:
#             print(f"Embeded {i*100/LEN:.2f}% of the full corpus")

    vect_pad = tf.keras.utils.pad_sequences(
        vect, truncating="post", padding="post", maxlen=256, dtype=float
    )

    return vect_pad

In [49]:
def write_tfrecords(X,vect_pad,y,filename):

    with tf.io.TFRecordWriter(
        os.path.join(os.environ["PATH_DATA"], f"data_{filename}.tfrecord")
    ) as writer:
        for j, i in enumerate(range(len(X))):
            example = serialize_example(
                X.index[i],
                X.text.str.encode("utf-8").iloc[i],
                vect_pad[i],
                X.nsentences.iloc[i],
                X.mean_w_p_s.iloc[i],
                X.var_w_p_s.iloc[i],
                y.label.iloc[i],
            )
            writer.write(example)
#             if j % 100 == 0:
#                 print(f"Wrote {i*100/len(X):.2f}% of the dataset on disk")
                

In [50]:
def chunkify(X,y,n):
    return [(X[i:i+n],y[i:i+n]) for i in range(0,len(X),n)]

In [51]:
def wraper(X,y,filename,wv):
    vect_pad = embed_corpus(X,wv)
    write_tfrecords(X,vect_pad,y,filename)
    print()

In [52]:
lazy_results=[]
chunk_n = 1
wv = load('glove-wiki-gigaword-100')        
for chunk in chunkify(X[:100],y[:100], 10):
    print(chunk_n)
    chunk_x,chunk_y=chunk[0],chunk[1]
    r = dask.delayed(wraper)(chunk_x,chunk_y,f'dataset_{chunk_n}',wv)
    lazy_results.append(r)
    chunk_n += 1

1
2
3
4
5
6
7
8
9
10


In [46]:
from dask.diagnostics import ProgressBar

In [53]:
with ProgressBar():
    dask.compute(*lazy_results)

[                                        ] | 0% Completed | 210.67 ms







[################################        ] | 80% Completed | 313.86 ms

[########################################] | 100% Completed | 421.87 ms


## full ds

In [ ]:
#useless - for tf.dataset usecase

ds = tf.data.Dataset.from_tensor_slices((X[:100].index,
                                        X[:100].text,
                                         vect_pad,
                                         X[:100].nsentences,
                                         X[:100].mean_w_p_s,
                                         X[:100].var_w_p_s,
                                         y[:100].label))

def tf_serialize_example(f0,f1,f2,f3,y):
    tf_string = tf.py_function(
    serialize_example,
    (f0, f1, f2, f3,y),  # Pass these args to the above function.
    tf.string)      # The return type is `tf.string`.
    return tf.reshape(tf_string, ()) # The result is a scalar.




def generator():
    for features in ds:
        yield serialize_example(*features)

serialized_features_dataset = tf.data.Dataset.from_generator(
    generator, output_types=tf.string, output_shapes=())


In [ ]:
## Pur python

In [ ]:
with tf.io.TFRecordWriter('hey.tfrecord') as writer:
    for i in range(100):
        example = serialize_example(
            X.index[i],
#             X.reset_index().id[i],
            X.text.str.encode('utf-8').iloc[i],
            vect_pad[i],
            X.nsentences.iloc[i],
            X.mean_w_p_s.iloc[i],
            X.var_w_p_s.iloc[i],
            y.label.iloc[i])
        writer.write(example)

In [129]:
#test reading
filenames = [f'../raw_data/bertds_{i}.tfrecord' for i in range(1,11)]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [130]:
a=0
for i in raw_dataset:
    example = tf.train.Example()
    example.ParseFromString(i.numpy())
    a+=1

In [131]:
a

100

In [132]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

<MapDataset element_spec=({'id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'mean_w_p_s': TensorSpec(shape=(), dtype=tf.float32, name=None), 'nsentences': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None), 'var_w_p_s': TensorSpec(shape=(), dtype=tf.float32, name=None), 'vect': TensorSpec(shape=<unknown>, dtype=tf.float64, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [133]:
next(iter(parsed_dataset))

InvalidArgumentError: Type mismatch between parsed tensor (float) and dtype (double)
	 [[{{node ParseTensor}}]] [Op:IteratorGetNext]

In [2]:
from google.cloud import storage

In [143]:
tfrecords_done = list_blobs('thebattle',extension='tfrecord')

In [144]:
tfrecords_done

['gs://thebattle/test/dataset_104.tfrecord',
 'gs://thebattle/testw2v_ds/dataset_123.tfrecord',
 'gs://thebattle/trainw2v_ds/dataset_123.tfrecord',
 'gs://thebattle/valw2v_ds/dataset_123.tfrecord',
 'gs://thebattle/w2v_ds/dataset_1.tfrecord',
 'gs://thebattle/w2v_ds/dataset_10.tfrecord',
 'gs://thebattle/w2v_ds/dataset_100.tfrecord',
 'gs://thebattle/w2v_ds/dataset_101.tfrecord',
 'gs://thebattle/w2v_ds/dataset_102.tfrecord',
 'gs://thebattle/w2v_ds/dataset_103.tfrecord',
 'gs://thebattle/w2v_ds/dataset_104.tfrecord',
 'gs://thebattle/w2v_ds/dataset_105.tfrecord',
 'gs://thebattle/w2v_ds/dataset_106.tfrecord',
 'gs://thebattle/w2v_ds/dataset_107.tfrecord',
 'gs://thebattle/w2v_ds/dataset_108.tfrecord',
 'gs://thebattle/w2v_ds/dataset_109.tfrecord',
 'gs://thebattle/w2v_ds/dataset_11.tfrecord',
 'gs://thebattle/w2v_ds/dataset_110.tfrecord',
 'gs://thebattle/w2v_ds/dataset_111.tfrecord',
 'gs://thebattle/w2v_ds/dataset_112.tfrecord',
 'gs://thebattle/w2v_ds/dataset_113.tfrecord',
 'gs://

In [3]:
def list_blobs(
    bucket_name: str,
    prefix: str = None,
    delimiter: str = None,
    extension: str = "tfrecord.gz",
) -> list:
    """List all files in a bucket, possibly filtered with extension.

    Parameters
    ----------
    bucket_name : str
        Name of the bucket the files are in
    prefix : str, optional
        prefix of the files, / indicates a path if specified in the delimiter `
        argument, by default None
    delimiter : str, optional
        character to be interpreted as a path delimiter, if present in the
        prefix, by default None
    extension : str, optional
        If provided, the function returns only files with this extension,
        by default "tfrecord.gz"

    Returns
    -------
    List
        List of all file names meeting given conditions.
    """

    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    file_names = [
        f"gs://{bucket_name}/{blob.name}"
        for blob in blobs
        if blob.name.endswith(extension)
    ]
    return file_names

In [16]:
blobs = list_blobs('thebattle',extension='tfrecord')

In [101]:
def move_blob(bucket_name, blob_name, destination_bucket_name, destination_blob_name,):
    """Moves a blob from one bucket to another with a new name."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The ID of your GCS object
    # blob_name = "your-object-name"
    # The ID of the bucket to move the object to
    # destination_bucket_name = "destination-bucket-name"
    # The ID of your new GCS object (optional)
    # destination_blob_name = "destination-object-name"

    storage_client = storage.Client()

    source_bucket = storage_client.bucket(bucket_name)
    source_blob = source_bucket.blob(blob_name)
    destination_bucket = storage_client.bucket(destination_bucket_name)
    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    # There is also an `if_source_generation_match` parameter, which is not used in this example.
    destination_generation_match_precondition = 1

    blob_copy = source_bucket.copy_blob(
        source_blob, destination_bucket, destination_blob_name,
    )
#     source_bucket.delete_blob(blob_name)

    print(
        "Blob {} in bucket {} moved to blob {} in bucket {}.".format(
            source_blob.name,
            source_bucket.name,
            blob_copy.name,
            destination_bucket.name,
        )
    )

In [90]:
np.random.shuffle(blobs)
nblobs = len(blobs)
ntrain = int(nblobs*.6)
nval = int(nblobs*.2)
ntest = int(nblobs*.2)
# train_blobs = blobs[:]
ntrain,ntest,nval

(180, 60, 60)

In [91]:
train_blobs = blobs[:ntrain]
val_blobs = blobs[ntrain:nval+ntrain]
test_blobs = blobs[ntrain+nval:]

In [92]:
'/'.join(train_blobs[0].split('/')[-2:])

'w2v_ds/dataset_123.tfrecord'

In [93]:
blobname = '/'.join(train_blobs[0].split('/')[-2:])
blobname
blobtest = 'haha/'+blobname
blobname,'gs://thebattle/w2v_ds/dataset_104.tfrecord'

('w2v_ds/dataset_123.tfrecord', 'gs://thebattle/w2v_ds/dataset_104.tfrecord')

In [94]:
blobtest

'haha/w2v_ds/dataset_123.tfrecord'

In [102]:
train_blobs

['gs://thebattle/w2v_ds/dataset_123.tfrecord',
 'gs://thebattle/w2v_ds/dataset_208.tfrecord',
 'gs://thebattle/w2v_ds/dataset_91.tfrecord',
 'gs://thebattle/w2v_ds/dataset_248.tfrecord',
 'gs://thebattle/w2v_ds/dataset_279.tfrecord',
 'gs://thebattle/w2v_ds/dataset_285.tfrecord',
 'gs://thebattle/w2v_ds/dataset_11.tfrecord',
 'gs://thebattle/w2v_ds/dataset_36.tfrecord',
 'gs://thebattle/w2v_ds/dataset_85.tfrecord',
 'gs://thebattle/w2v_ds/dataset_235.tfrecord',
 'gs://thebattle/w2v_ds/dataset_55.tfrecord',
 'gs://thebattle/w2v_ds/dataset_67.tfrecord',
 'gs://thebattle/w2v_ds/dataset_152.tfrecord',
 'gs://thebattle/w2v_ds/dataset_112.tfrecord',
 'gs://thebattle/w2v_ds/dataset_13.tfrecord',
 'gs://thebattle/w2v_ds/dataset_293.tfrecord',
 'gs://thebattle/w2v_ds/dataset_26.tfrecord',
 'gs://thebattle/w2v_ds/dataset_287.tfrecord',
 'gs://thebattle/w2v_ds/dataset_30.tfrecord',
 'gs://thebattle/w2v_ds/dataset_2.tfrecord',
 'gs://thebattle/w2v_ds/dataset_258.tfrecord',
 'gs://thebattle/w2v_ds/

In [106]:
destblob = b.split('/')[-2]+'/'+bn+'/'+b.split('/')[-1]

In [107]:
destblob

'w2v_ds/val/dataset_277.tfrecord'

In [114]:
tf.distribute.Strategy

tensorflow.python.distribute.distribute_lib.Strategy

In [116]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [117]:
tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
for blob,bn in zip([train_blobs,test_blobs,val_blobs],['train','test','val']):
    for b in blob:
        try:
            sourceblob = '/'.join(b.split('/')[-2:])
            destblob = b.split('/')[-2]+'/'+bn+'/'+b.split('/')[-1]
            move_blob('thebattle',sourceblob,'thebattle',destblob)
        except:
            'oups'

In [23]:
raw_train = tf.data.TFRecordDataset(train_blobs)
raw_val = tf.data.TFRecordDataset(val_blobs)
raw_test = tf.data.TFRecordDataset(test_blobs)

In [137]:
feature_description = {
    'id': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'text': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'vect': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'nsentences': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'mean_w_p_s': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
    'var_w_p_s': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
    'label': tf.io.FixedLenFeature([], tf.int64, default_value=0),

}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
    parsed_ex = tf.io.parse_single_example(example_proto, feature_description)
    label = parsed_ex.pop('label')
    vect = parsed_ex.pop('vect')
    parsed_ex['vect'] = tf.io.parse_tensor(vect,tf.float32)
    return parsed_ex,label

In [139]:
next(iter(raw_dataset.map(_parse_function).take(1)))

({'id': <tf.Tensor: shape=(), dtype=int64, numpy=63064638>,
  'mean_w_p_s': <tf.Tensor: shape=(), dtype=float32, numpy=13.0>,
  'nsentences': <tf.Tensor: shape=(), dtype=int64, numpy=6>,
  'text': <tf.Tensor: shape=(), dtype=string, numpy=b'Sexhow railway station was a railway station located in the town of Sexhow, on the Cumbrian Coast Line in North West England. The station was opened by the Lancashire and Yorkshire Railway on 7 October 1870. It was closed to passengers on 5 January 1950, and to goods on 12 May 1965. \n\nThe station building is now a private residence. There is a small amount of trackage remaining near the building, used currently by a local agricultural business.'>,
  'var_w_p_s': <tf.Tensor: shape=(), dtype=float32, numpy=7.187953>,
  'vect': <tf.Tensor: shape=(768,), dtype=float32, numpy=
  array([ 0.32985756,  0.6226763 ,  0.8880232 , -0.6134216 , -0.74101526,
          0.76420176, -0.61633563, -0.4643601 ,  0.6030929 , -0.68834907,
         -0.00301795, -0.28466

In [148]:
tfrecords_done = [
    f for f in os.listdir(os.environ["PATH_DATA"]) if f.startswith("bertds")
]

In [149]:
tfrecords_done

['bertds_10.tfrecord',
 'bertds_7.tfrecord',
 'bertds_6.tfrecord',
 'bertds_1.tfrecord',
 'bertds_4.tfrecord',
 'bertds_5.tfrecord',
 'bertds_3.tfrecord',
 'bertds_2.tfrecord',
 'bertds_9.tfrecord',
 'bertds_8.tfrecord']

In [159]:
def chunkify(X, y, n):
    return [(X[i : i + n], y[i : i + n]) for i in range(0, len(X), n)]

In [160]:
lazy_results = []
chunk_n = 1


In [164]:
"bertds_{chunk_n}.tfrecord" in tfrecords_done

True

In [168]:
chunk_n = 1
not_done = []
done = []
for chunk in chunkify(X, y, 10):
    if f"bertds_{chunk_n}.tfrecord" in tfrecords_done:
        print('haha ! skipped !',chunk_n)
        chunk_n += 1
        continue
    print('hmmm not done it seems')
    chunk_n +=1 

haha ! skipped ! 1
haha ! skipped ! 2
haha ! skipped ! 3
haha ! skipped ! 4
haha ! skipped ! 5
haha ! skipped ! 6
haha ! skipped ! 7
haha ! skipped ! 8
haha ! skipped ! 9
haha ! skipped ! 10
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm not done it seems
hmmm